In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_theme()

import re

from IPython.display import display

In [2]:
base_dir = '/Users/das/work/svn/Gits/a_Projects/101_clembench/clembench-runs'

In [3]:
results_path = base_dir + '/v1.6_multimodal/results.csv'
raw_path = base_dir + '/v1.6_multimodal/raw.csv'

res_df = pd.read_csv(results_path)
raw_df = pd.read_csv(raw_path)


def parse_model_names(in_df, col_pos=0):
    in_df.loc[:,'model'] = in_df.iloc[:, col_pos].str.replace(r'(\w*)-t0.0.*', r'\1').str.lower()
    in_df.loc[in_df.model == 'openchat_3.5', 'model'] = 'openchat-3.5'
    return in_df

res_df = parse_model_names(res_df)
raw_df = parse_model_names(raw_df, col_pos=2)

/var/folders/v4/sxby0fb08xjbdx001s6j3wrh0000gq/T/ipykernel_75004/3106035806.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  in_df.loc[:,'model'] = in_df.iloc[:, col_pos].str.replace(r'(\w*)-t0.0.*', r'\1').str.lower()
/var/folders/v4/sxby0fb08xjbdx001s6j3wrh0000gq/T/ipykernel_75004/3106035806.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  in_df.loc[:,'model'] = in_df.iloc[:, col_pos].str.replace(r'(\w*)-t0.0.*', r'\1').str.lower()


In [4]:
raw_df

,Unnamed: 0,game,model,experiment,episode,metric,value
0,0,matchit_1q,gpt-4-1106-vision-preview,1_similar_image,episode_4,Violated Request Count,0.0
1,1,matchit_1q,gpt-4-1106-vision-preview,1_similar_image,episode_4,Parsed Request Count,8.0
2,2,matchit_1q,gpt-4-1106-vision-preview,1_similar_image,episode_4,Request Count,8.0
3,3,matchit_1q,gpt-4-1106-vision-preview,1_similar_image,episode_4,Aborted,0.0
4,4,matchit_1q,gpt-4-1106-vision-preview,1_similar_image,episode_4,Success,1.0
...,...,...,...,...,...,...,...
128793,128793,matchit_ascii,llava-v1.6-vicuna-13b-hf,1_similar_grid_1,episode_9,Played,1.0
128794,128794,matchit_ascii,llava-v1.6-vicuna-13b-hf,1_similar_grid_1,episode_7,Played,1.0
128795,128795,matchit_ascii,llava-v1.6-vicuna-13b-hf,1_similar_grid_1,episode_1,Played,1.0
128796,128796,matchit_ascii,llava-v1.6-vicuna-13b-hf,1_similar_grid_1,episode_6,Played,1.0


## Doing stuff with results.csv

### Producing the overall results table

In [5]:
res_df

,Unnamed: 0,"-, clemscore","all, Average % Played","all, Average Quality Score","matchit, % Played","matchit, Quality Score","matchit, Quality Score (std)","matchit_1q, % Played","matchit_1q, Quality Score","matchit_1q, Quality Score (std)",...,"textmapworld_graphreasoning, % Played","textmapworld_graphreasoning, Quality Score","textmapworld_graphreasoning, Quality Score (std)","textmapworld_questions, % Played","textmapworld_questions, Quality Score","textmapworld_questions, Quality Score (std)","textmapworld_specificroom, % Played","textmapworld_specificroom, Quality Score","textmapworld_specificroom, Quality Score (std)",model
0,claude-3-opus-20240229-t0.0--claude-3-opus-202...,74.51,99.72,74.72,100.00,81.67,39.02,100.00,80.00,40.34,...,100.00,76.69,12.80,100.00,68.44,19.88,100.00,100.00,0.00,claude-3-opus-20240229
1,gemini-1.5-flash-latest-t0.0--gemini-1.5-flash...,54.79,78.94,69.41,85.00,84.31,36.73,95.00,87.72,33.11,...,0.00,NaN,NaN,63.33,48.07,7.87,53.33,100.00,0.00,gemini-1.5-flash-latest
2,gemini-1.5-pro-latest-t0.0--gemini-1.5-pro-lat...,69.63,100.00,69.63,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gemini-1.5-pro-latest
3,gpt-4-1106-vision-preview-t0.0--gpt-4-1106-vis...,74.90,99.07,75.60,100.00,80.00,40.34,100.00,85.00,36.01,...,100.00,67.46,15.11,100.00,66.11,16.35,96.67,100.00,0.00,gpt-4-1106-vision-preview
4,gpt-4o-2024-05-13-t0.0--gpt-4o-2024-05-13-t0.0,75.65,91.82,82.39,100.00,78.33,41.55,100.00,81.67,39.02,...,96.67,66.12,12.83,83.33,63.53,17.57,96.67,100.00,0.00,gpt-4o-2024-05-13
5,idefics-80b-instruct-t0.0--idefics-80b-instruc...,24.54,56.15,43.70,88.14,55.77,50.15,86.44,58.82,49.71,...,NaN,NaN,NaN,10.00,47.85,6.03,0.00,NaN,NaN,idefics-80b-instruct
6,idefics-9b-instruct-t0.0--idefics-9b-instruct-...,16.67,47.73,34.93,100.00,33.33,47.54,100.00,33.33,47.54,...,0.00,NaN,NaN,0.00,NaN,NaN,0.00,NaN,NaN,idefics-9b-instruct
7,llava-1.5-13b-hf-t0.0--llava-1.5-13b-hf-t0.0,23.44,52.06,45.03,100.00,66.67,47.54,100.00,66.67,47.54,...,0.00,NaN,NaN,3.33,40.00,NaN,20.00,83.33,40.82,llava-1.5-13b-hf
8,llava-v1.6-34b-hf-t0.0--llava-v1.6-34b-hf-t0.0,30.65,51.54,59.46,90.00,74.07,44.23,100.00,76.67,42.65,...,0.00,NaN,NaN,0.00,NaN,NaN,26.67,100.00,0.00,llava-v1.6-34b-hf
9,llava-v1.6-vicuna-13b-hf-t0.0--llava-v1.6-vicu...,28.53,43.44,65.68,60.00,63.89,48.71,98.33,69.49,46.44,...,0.00,NaN,NaN,0.00,NaN,NaN,3.33,100.00,NaN,llava-v1.6-vicuna-13b-hf


In [6]:
res_df.columns

Index(['Unnamed: 0', '-, clemscore', 'all, Average % Played',
       'all, Average Quality Score', 'matchit, % Played',
       'matchit, Quality Score', 'matchit, Quality Score (std)',
       'matchit_1q, % Played', 'matchit_1q, Quality Score',
       'matchit_1q, Quality Score (std)', 'matchit_5q, % Played',
       'matchit_5q, Quality Score', 'matchit_5q, Quality Score (std)',
       'matchit_ascii, % Played', 'matchit_ascii, Quality Score',
       'matchit_ascii, Quality Score (std)', 'matchit_ascii_1q, % Played',
       'matchit_ascii_1q, Quality Score',
       'matchit_ascii_1q, Quality Score (std)', 'matchit_ascii_5q, % Played',
       'matchit_ascii_5q, Quality Score',
       'matchit_ascii_5q, Quality Score (std)', 'matchit_ascii_info, % Played',
       'matchit_ascii_info, Quality Score',
       'matchit_ascii_info, Quality Score (std)', 'matchit_info, % Played',
       'matchit_info, Quality Score', 'matchit_info, Quality Score (std)',
       'mm_mapworld, % Played', 'mm_mapw

double checking... where are the averages coming from?

In [7]:
res_df.iloc[:, 4:-1:3].mean(axis=1)

0     99.719474
1     78.938421
2    100.000000
3     99.068947
4     91.824737
5     56.150000
6     47.729474
7     52.055000
8     51.537222
9     43.443889
dtype: float64

Ah ok, this is indeed averaged just across the rows. So the games are weighted heavily differently, depending on how they are split up.

In [8]:
res_df.iloc[:, [-1,1,2,3]].sort_values(by='-, clemscore', ascending=False).drop(2)

,model,"-, clemscore","all, Average % Played","all, Average Quality Score"
4,gpt-4o-2024-05-13,75.65,91.82,82.39
3,gpt-4-1106-vision-preview,74.90,99.07,75.60
0,claude-3-opus-20240229,74.51,99.72,74.72
1,gemini-1.5-flash-latest,54.79,78.94,69.41
8,llava-v1.6-34b-hf,30.65,51.54,59.46
9,llava-v1.6-vicuna-13b-hf,28.53,43.44,65.68
5,idefics-80b-instruct,24.54,56.15,43.70
7,llava-1.5-13b-hf,23.44,52.06,45.03
6,idefics-9b-instruct,16.67,47.73,34.93


In [9]:
(
    res_df.iloc[:, [-1,1,2,3]].sort_values(by='-, clemscore', ascending=False)
    .drop(2)
    .to_latex('Out/overall_results.tex',
              index=False,
              header=['model', 'clemscore', '% played', 'quality']
             )
)

/var/folders/v4/sxby0fb08xjbdx001s6j3wrh0000gq/T/ipykernel_75004/1392124316.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  res_df.iloc[:, [-1,1,2,3]].sort_values(by='-, clemscore', ascending=False)


Ok, what I could do is I could calculate new averages without the ascii games. Should be easy.

In [10]:
{'matchit' + suffix: 'matchit_3q' + suffix 
             for suffix in [', % Played', ' Quality Score', 'Quality Score (std)']}

{'matchit, % Played': 'matchit_3q, % Played',
 'matchit Quality Score': 'matchit_3q Quality Score',
 'matchitQuality Score (std)': 'matchit_3qQuality Score (std)'}

In [11]:
smaller_res_df = \
(
    res_df.loc[:,~res_df.columns.str.contains(r'text|ascii')]
    .rename({'matchit' + suffix: 'matchit_3q' + suffix 
             for suffix in [', % Played', ', Quality Score', ', Quality Score (std)']}, axis=1)
    .sort_index(axis=1)
    .set_index('model')
    .drop('Unnamed: 0', axis=1)
)

smaller_res_df = \
(
    smaller_res_df.loc[:, ~smaller_res_df.columns.str.contains('std')]
    .drop('gemini-1.5-pro-latest')
    .sort_values(by='-, clemscore', ascending=False)
)

smaller_res_df

,"-, clemscore","all, Average % Played","all, Average Quality Score","matchit_1q, % Played","matchit_1q, Quality Score","matchit_3q, % Played","matchit_3q, Quality Score","matchit_5q, % Played","matchit_5q, Quality Score","matchit_info, % Played",...,"mm_mapworld_graphs, % Played","mm_mapworld_graphs, Quality Score","mm_mapworld_qa, % Played","mm_mapworld_qa, Quality Score","mm_mapworld_specificroom, % Played","mm_mapworld_specificroom, Quality Score","multimodal_referencegame, % Played","multimodal_referencegame, Quality Score","referencegame, % Played","referencegame, Quality Score"
model,,,,,,,,,,,,,,,,,,,,,
gpt-4o-2024-05-13,75.65,91.82,82.39,100.00,81.67,100.00,78.33,100.00,81.67,100.00,...,90.00,76.06,43.33,70.23,96.67,93.10,100.00,75.58,100.00,90.00
gpt-4-1106-vision-preview,74.90,99.07,75.60,100.00,85.00,100.00,80.00,100.00,90.00,100.00,...,90.00,77.25,96.67,59.48,100.00,76.67,98.97,68.57,100.00,29.44
claude-3-opus-20240229,74.51,99.72,74.72,100.00,80.00,100.00,81.67,100.00,80.00,100.00,...,96.67,85.12,100.00,66.97,100.00,53.33,100.00,47.18,100.00,29.44
gemini-1.5-flash-latest,54.79,78.94,69.41,95.00,87.72,85.00,84.31,55.00,87.88,71.67,...,46.67,62.72,100.00,48.94,93.33,32.14,100.00,41.54,100.00,61.11
llava-v1.6-34b-hf,30.65,51.54,59.46,100.00,76.67,90.00,74.07,86.67,71.15,95.00,...,0.00,NaN,0.00,NaN,3.33,100.00,NaN,NaN,100.00,36.67
llava-v1.6-vicuna-13b-hf,28.53,43.44,65.68,98.33,69.49,60.00,63.89,58.33,65.71,56.67,...,3.33,75.63,3.33,37.50,26.67,62.50,NaN,NaN,100.00,32.22
idefics-80b-instruct,24.54,56.15,43.70,86.44,58.82,88.14,55.77,93.22,47.27,88.14,...,50.00,81.36,3.57,0.00,33.33,50.00,100.00,33.59,100.00,31.11
llava-1.5-13b-hf,23.44,52.06,45.03,100.00,66.67,100.00,66.67,100.00,63.33,98.33,...,0.00,NaN,0.00,NaN,3.33,0.00,NaN,NaN,100.00,35.00
idefics-9b-instruct,16.67,47.73,34.93,100.00,33.33,100.00,33.33,100.00,33.33,100.00,...,0.00,NaN,0.00,NaN,0.00,NaN,90.00,31.34,12.78,30.43


In [12]:
smaller_res_df.iloc[:, 1] = smaller_res_df.iloc[:, 3:-1:2].mean(axis=1)
smaller_res_df.iloc[:, 2] = smaller_res_df.iloc[:, 4:-1:2].mean(axis=1)
smaller_res_df.iloc[:, 0] = (smaller_res_df.iloc[:,1] / 100) * smaller_res_df.iloc[:,2]
smaller_res_df.sort_values(by='-, clemscore', ascending=False)

,"-, clemscore","all, Average % Played","all, Average Quality Score","matchit_1q, % Played","matchit_1q, Quality Score","matchit_3q, % Played","matchit_3q, Quality Score","matchit_5q, % Played","matchit_5q, Quality Score","matchit_info, % Played",...,"mm_mapworld_graphs, % Played","mm_mapworld_graphs, Quality Score","mm_mapworld_qa, % Played","mm_mapworld_qa, Quality Score","mm_mapworld_specificroom, % Played","mm_mapworld_specificroom, Quality Score","multimodal_referencegame, % Played","multimodal_referencegame, Quality Score","referencegame, % Played","referencegame, Quality Score"
model,,,,,,,,,,,,,,,,,,,,,
gpt-4-1106-vision-preview,76.373958,98.564000,77.486667,100.00,85.00,100.00,80.00,100.00,90.00,100.00,...,90.00,77.25,96.67,59.48,100.00,76.67,98.97,68.57,100.00,29.44
claude-3-opus-20240229,72.173304,99.667000,72.414444,100.00,80.00,100.00,81.67,100.00,80.00,100.00,...,96.67,85.12,100.00,66.97,100.00,53.33,100.00,47.18,100.00,29.44
gpt-4o-2024-05-13,69.275220,88.200000,78.543333,100.00,81.67,100.00,78.33,100.00,81.67,100.00,...,90.00,76.06,43.33,70.23,96.67,93.10,100.00,75.58,100.00,90.00
gemini-1.5-flash-latest,56.068369,84.667000,66.222222,95.00,87.72,85.00,84.31,55.00,87.88,71.67,...,46.67,62.72,100.00,48.94,93.33,32.14,100.00,41.54,100.00,61.11
llava-v1.6-34b-hf,42.681389,52.777778,80.870000,100.00,76.67,90.00,74.07,86.67,71.15,95.00,...,0.00,NaN,0.00,NaN,3.33,100.00,NaN,NaN,100.00,36.67
idefics-80b-instruct,30.449397,66.284000,45.937778,86.44,58.82,88.14,55.77,93.22,47.27,88.14,...,50.00,81.36,3.57,0.00,33.33,50.00,100.00,33.59,100.00,31.11
llava-1.5-13b-hf,28.349364,55.740000,50.860000,100.00,66.67,100.00,66.67,100.00,63.33,98.33,...,0.00,NaN,0.00,NaN,3.33,0.00,NaN,NaN,100.00,35.00
llava-v1.6-vicuna-13b-hf,28.187049,46.295556,60.885000,98.33,69.49,60.00,63.89,58.33,65.71,56.67,...,3.33,75.63,3.33,37.50,26.67,62.50,NaN,NaN,100.00,32.22
idefics-9b-instruct,16.557551,50.278000,32.932000,100.00,33.33,100.00,33.33,100.00,33.33,100.00,...,0.00,NaN,0.00,NaN,0.00,NaN,90.00,31.34,12.78,30.43


Ok, actually, I want averages per game, and then averages over that.

So, the following table averages across all variants of a game, including the ASCII representations, to get an average per game. Then it averages these to get an overall average. I.e., each game is weighted equally in the final score.

In [13]:
out_df = pd.DataFrame()
out_df.loc[:, 'model'] = res_df['model']

game_families = ['matchit', 'mapworld', 'referencegame']
for game in game_families:
    played = \
    (
        res_df
        .loc[:,res_df.columns.str.contains(game)] # & ~res_df.columns.str.contains('matchit'))]
        .iloc[:,0:-1:3]
    ).mean(axis=1)

    qualit = \
    (
        res_df
        .loc[:,res_df.columns.str.contains(game)] # & ~res_df.columns.str.contains('matchit'))]
        .iloc[:,1:-1:3]
    ).mean(axis=1)

    out_df.loc[:, game + ' % played'] = played
    out_df.loc[:, game + ' quality'] = qualit
    
    
out_df.loc[:, 'avg % played'] = out_df.iloc[:,1:-1:2].mean(axis=1)
out_df.loc[:, 'avg quality'] = out_df.iloc[:,2:-1:2].mean(axis=1)
out_df.loc[:, 'clemscore'] = out_df['avg % played'] / 100 * out_df['avg quality']
out_df = out_df.set_index('model').drop('gemini-1.5-pro-latest')
out_df = out_df.loc[:, ['clemscore', 'avg % played', 'avg quality',
                        'matchit % played', 'matchit quality',
                        'mapworld % played', 'mapworld quality',
                        'referencegame % played', 'referencegame quality'] ]
out_df.sort_values(by='clemscore', ascending=False)

,clemscore,avg % played,avg quality,matchit % played,matchit quality,mapworld % played,mapworld quality,referencegame % played,referencegame quality
model,,,,,,,,,
gpt-4o-2024-05-13,77.964727,94.247037,82.723796,100.00000,88.54250,82.741111,76.838889,100.000,82.790
gpt-4-1106-vision-preview,68.479908,99.211296,69.024306,100.00000,84.27125,98.148889,73.796667,99.485,49.005
claude-3-opus-20240229,65.428454,99.802593,65.557870,100.00000,82.29250,99.407778,76.071111,100.000,38.310
gemini-1.5-flash-latest,54.907119,84.619491,64.887083,84.89625,83.37500,68.962222,59.961250,100.000,51.325
llava-v1.6-34b-hf,38.688138,67.560324,57.264583,96.45875,54.29375,6.222222,80.830000,100.000,36.670
llava-v1.6-vicuna-13b-hf,34.142893,61.431713,55.578611,76.66625,72.95250,7.628889,61.563333,100.000,32.220
llava-1.5-13b-hf,28.573063,67.936343,42.058583,97.29125,46.49375,6.517778,44.682000,100.000,35.000
idefics-80b-instruct,27.525872,67.112500,41.014524,84.72500,43.35500,16.612500,47.338571,100.000,32.350
idefics-9b-instruct,17.425288,50.614444,34.427500,100.00000,35.72750,0.453333,36.670000,51.390,30.885


In [14]:
(
    out_df
    .sort_values(by='clemscore', ascending=False)
    .to_latex('Out/results_per_game_macro_avg.tex',
                    header=['clscr', 'avg %p', 'avg ql', 'mt %p', 'mt ql', 'mp %p', 'mp ql', 'ref %p', 'ref ql'],
                    column_format='Trrr|rr|rr|rr',
                    float_format="%.2f")
)

/var/folders/v4/sxby0fb08xjbdx001s6j3wrh0000gq/T/ipykernel_75004/1162459983.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  out_df


**^^^  This table is in the paper**

### By game, all variants

I guess the logical next step would be to produce a table that has averages by game, and then all variants...

In [15]:
smaller_res_df = \
(
    res_df   # .loc[:,~res_df.columns.str.contains(r'text|ascii')]
    .rename({'matchit' + suffix: 'matchit_3q' + suffix 
             for suffix in [', % Played', ', Quality Score', ', Quality Score (std)']}, axis=1)
    .sort_index(axis=1)
    .set_index('model')
    .drop('Unnamed: 0', axis=1)
)

smaller_res_df = \
(
    smaller_res_df.loc[:, ~smaller_res_df.columns.str.contains('std')]
    .drop('gemini-1.5-pro-latest')
    .sort_values(by='-, clemscore', ascending=False)
)

#smaller_res_df

game_families = ['matchit', 'mapworld', 'referencegame']
game_dfs = []
for game in game_families:
    played = \
    (
        smaller_res_df
        .loc[:,smaller_res_df.columns.str.contains(game)] # & ~res_df.columns.str.contains('matchit'))]
        .iloc[:,0:-1:2]
    ).mean(axis=1)

    qualit = \
    (
        smaller_res_df
        .loc[:,smaller_res_df.columns.str.contains(game)] # & ~res_df.columns.str.contains('matchit'))]
        .iloc[:,1::2]
    ).mean(axis=1)


    smaller_res_df.loc[:, game + ' cs'] = played / 100 * qualit
    smaller_res_df.loc[:, game + ' %p'] = played
    smaller_res_df.loc[:, game + ' ql'] = qualit

    collist = [game + ' cs', game + ' %p', game + ' ql'] +  \
        list(smaller_res_df.columns[smaller_res_df.columns.str.contains(game)])
    
    display(smaller_res_df.loc[:, collist[:-3]])
    
    header = [re.sub(game + r'.(.*)', r'\1', col) for col in collist[:-3]]
    header = [re.sub(r'(.*) % Played', r'\1 %p', col) for col in header]
    header = [re.sub(r'(.*) Quality Score', r'\1 ql', col) for col in header]
    
    #print(header)
    
    (
        smaller_res_df
        .loc[:, collist[:-3]]
        .to_latex('Out/results_' + game + '.tex',
                        header=header,
                        #column_format='Trrr|rr|rr|rr',
                        float_format="%.2f")
    )

    


#smaller_res_df

,matchit cs,matchit %p,matchit ql,"matchit_1q, % Played","matchit_1q, Quality Score","matchit_3q, % Played","matchit_3q, Quality Score","matchit_5q, % Played","matchit_5q, Quality Score","matchit_ascii, % Played","matchit_ascii, Quality Score","matchit_ascii_1q, % Played","matchit_ascii_1q, Quality Score","matchit_ascii_5q, % Played","matchit_ascii_5q, Quality Score","matchit_ascii_info, % Played","matchit_ascii_info, Quality Score","matchit_info, % Played","matchit_info, Quality Score"
model,,,,,,,,,,,,,,,,,,,
gpt-4o-2024-05-13,88.542500,100.00000,88.54250,100.00,81.67,100.00,78.33,100.00,81.67,100.0,97.50,100.0,92.50,100.0,100.00,100.00,100.00,100.00,76.67
gpt-4-1106-vision-preview,84.271250,100.00000,84.27125,100.00,85.00,100.00,80.00,100.00,90.00,100.0,72.50,100.0,87.50,100.0,82.50,100.00,90.00,100.00,86.67
claude-3-opus-20240229,82.292500,100.00000,82.29250,100.00,80.00,100.00,81.67,100.00,80.00,100.0,85.00,100.0,82.50,100.0,77.50,100.00,90.00,100.00,81.67
gemini-1.5-flash-latest,70.782248,84.89625,83.37500,95.00,87.72,85.00,84.31,55.00,87.88,97.5,76.92,100.0,85.00,82.5,78.79,92.50,75.68,71.67,90.70
llava-v1.6-34b-hf,52.371073,96.45875,54.29375,100.00,76.67,90.00,74.07,86.67,71.15,100.0,30.00,100.0,37.50,100.0,32.50,100.00,30.00,95.00,82.46
llava-v1.6-vicuna-13b-hf,55.929946,76.66625,72.95250,98.33,69.49,60.00,63.89,58.33,65.71,77.5,80.65,100.0,85.00,77.5,80.65,85.00,82.35,56.67,55.88
idefics-80b-instruct,36.732524,84.72500,43.35500,86.44,58.82,88.14,55.77,93.22,47.27,80.0,37.50,77.5,35.48,90.0,30.56,74.36,27.59,88.14,53.85
llava-1.5-13b-hf,45.234351,97.29125,46.49375,100.00,66.67,100.00,66.67,100.00,63.33,92.5,27.03,100.0,32.50,90.0,22.22,97.50,35.90,98.33,57.63
idefics-9b-instruct,35.727500,100.00000,35.72750,100.00,33.33,100.00,33.33,100.00,33.33,100.0,37.50,100.0,22.50,100.0,45.00,100.00,47.50,100.00,33.33


/var/folders/v4/sxby0fb08xjbdx001s6j3wrh0000gq/T/ipykernel_75004/858091208.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  smaller_res_df


,mapworld cs,mapworld %p,mapworld ql,"mm_mapworld, % Played","mm_mapworld, Quality Score","mm_mapworld_graphs, % Played","mm_mapworld_graphs, Quality Score","mm_mapworld_qa, % Played","mm_mapworld_qa, Quality Score","mm_mapworld_specificroom, % Played",...,"textmapworld, % Played","textmapworld, Quality Score","textmapworld_description, % Played","textmapworld_description, Quality Score","textmapworld_graphreasoning, % Played","textmapworld_graphreasoning, Quality Score","textmapworld_questions, % Played","textmapworld_questions, Quality Score","textmapworld_specificroom, % Played","textmapworld_specificroom, Quality Score"
model,,,,,,,,,,,,,,,,,,,,,
gpt-4o-2024-05-13,63.577350,82.741111,76.838889,52.0,73.58,90.00,76.06,43.33,70.23,96.67,...,90.0,74.25,96.00,74.68,96.67,66.12,83.33,63.53,96.67,100.00
gpt-4-1106-vision-preview,72.430608,98.148889,73.796667,100.0,73.74,90.00,77.25,96.67,59.48,100.00,...,100.0,73.62,100.00,69.84,100.00,67.46,100.00,66.11,96.67,100.00
claude-3-opus-20240229,75.620601,99.407778,76.071111,100.0,75.79,96.67,85.12,100.00,66.97,100.00,...,100.0,83.83,98.00,74.47,100.00,76.69,100.00,68.44,100.00,100.00
gemini-1.5-flash-latest,41.350610,68.962222,59.961250,100.0,60.05,46.67,62.72,100.00,48.94,93.33,...,64.0,66.08,100.00,61.69,0.00,NaN,63.33,48.07,53.33,100.00
llava-v1.6-34b-hf,5.029422,6.222222,80.830000,0.0,NaN,0.00,NaN,0.00,NaN,3.33,...,0.0,NaN,26.00,42.49,0.00,NaN,0.00,NaN,26.67,100.00
llava-v1.6-vicuna-13b-hf,4.696598,7.628889,61.563333,10.0,56.48,3.33,75.63,3.33,37.50,26.67,...,0.0,NaN,22.00,37.27,0.00,NaN,0.00,NaN,3.33,100.00
idefics-80b-instruct,7.864120,16.612500,47.338571,20.0,32.78,50.00,81.36,3.57,0.00,33.33,...,6.0,58.71,10.00,60.67,NaN,NaN,10.00,47.85,0.00,NaN
llava-1.5-13b-hf,2.912273,6.517778,44.682000,0.0,NaN,0.00,NaN,0.00,NaN,3.33,...,10.0,48.90,22.00,51.18,0.00,NaN,3.33,40.00,20.00,83.33
idefics-9b-instruct,0.166237,0.453333,36.670000,0.0,NaN,0.00,NaN,0.00,NaN,0.00,...,0.0,NaN,4.08,36.67,0.00,NaN,0.00,NaN,0.00,NaN


/var/folders/v4/sxby0fb08xjbdx001s6j3wrh0000gq/T/ipykernel_75004/858091208.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  smaller_res_df


,referencegame cs,referencegame %p,referencegame ql,"multimodal_referencegame, % Played","multimodal_referencegame, Quality Score","referencegame, % Played","referencegame, Quality Score"
model,,,,,,,
gpt-4o-2024-05-13,82.790000,100.000,82.790,100.00,75.58,100.00,90.00
gpt-4-1106-vision-preview,48.752624,99.485,49.005,98.97,68.57,100.00,29.44
claude-3-opus-20240229,38.310000,100.000,38.310,100.00,47.18,100.00,29.44
gemini-1.5-flash-latest,51.325000,100.000,51.325,100.00,41.54,100.00,61.11
llava-v1.6-34b-hf,36.670000,100.000,36.670,NaN,NaN,100.00,36.67
llava-v1.6-vicuna-13b-hf,32.220000,100.000,32.220,NaN,NaN,100.00,32.22
idefics-80b-instruct,32.350000,100.000,32.350,100.00,33.59,100.00,31.11
llava-1.5-13b-hf,35.000000,100.000,35.000,NaN,NaN,100.00,35.00
idefics-9b-instruct,15.871802,51.390,30.885,90.00,31.34,12.78,30.43


/var/folders/v4/sxby0fb08xjbdx001s6j3wrh0000gq/T/ipykernel_75004/858091208.py:54: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  smaller_res_df


### Do stuff with the raw results table

so far actually not used...

In [ ]:
raw_df.game.value_counts()

In [ ]:
list(raw_df.game.unique())

In [ ]:
{g:list(raw_df[raw_df.game == g].metric.unique()) for g in raw_df.game.unique()}    

Nice! Our foresight pays off; it looks like indeed every game has a 'Main Score' declared, and 'Played'.

In [ ]:
raw_df.experiment.value_counts()

In [ ]:
raw_df.metric.value_counts()

In [ ]:
raw_df[(raw_df.game == 'matchit_ascii') & 
       (raw_df.metric == 'Main Score') & 
       (raw_df.model == 'gpt-4o-2024-05-13')].value.mean()

In [ ]:
grouped = raw_df.groupby(['model', 'game'])

In [ ]:
list(grouped.groups.keys())[:10]

In [ ]:
grouped.get_group(('claude-3-opus-20240229', 'matchit'))

In [ ]:
raw_df.groupby(['model', 'game', 'metric'])['value'].mean()

In [ ]:
a = raw_df.groupby(['model', 'game', 'metric'])['value']

In [ ]:
a.count()

In [ ]:
raw_df[raw_df.metric == 'Main Score'].groupby(['model', 'game', 'metric'])['value'].mean()

In [ ]:
(
    raw_df[(raw_df.metric == 'Main Score') | (raw_df.metric == 'Played')]
    .groupby(['model', 'game', 'metric'])['value']
    .mean()
    .unstack(level=1)
    .rename({'matchit': 'matchit_3q', 'matchit_ascii': 'matchit_ascii_3q'}, axis=1)
    .sort_index(axis=1)
)

In [ ]:
res_tab = \
(
    raw_df[(raw_df.metric == 'Main Score') | (raw_df.metric == 'Played')]
    .groupby(['model', 'game', 'metric'])['value']
    .mean()
    .unstack(level=1)
    .rename({'matchit': 'matchit_3q', 'matchit_ascii': 'matchit_ascii_3q'}, axis=1)
    .sort_index(axis=1)
)

res_tab.loc[pd.IndexSlice[:, 'Played'], :] = res_tab.loc[pd.IndexSlice[:, 'Played'], :] * 100

res_tab = res_tab.drop('gemini-1.5-pro-latest')
res_tab

In [ ]:
res_tab.loc[:, res_tab.columns.str.contains('matchit')].mean(axis=1)

In [ ]:
game_families = ['matchit', 'mapworld', 'referencegame']
for game in game_families:
    res_tab.loc[:, game + '-avg'] = res_tab.loc[:, res_tab.columns.str.contains(game)].mean(axis=1)
res_tab.loc[:, [g + '-avg' for g in game_families]]